<a href="https://colab.research.google.com/github/imostafizur/Feedback_Analysis/blob/master/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [21]:
import os

os.environ['KAGGLE_USERNAME'] = "imostafizur" # username from the json file
os.environ['KAGGLE_KEY'] = "209f4b70995998e465b2702534f7a5ff" # key from the json file

In [22]:
!kaggle datasets download -d grikomsn/amazon-cell-phones-reviews

amazon-cell-phones-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
!unzip amazon-cell-phones-reviews.zip

Archive:  amazon-cell-phones-reviews.zip
replace 20191226-items.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [32]:
df0 = pd.read_csv('20191226-items.csv')

In [33]:
# create a new dataframe with columns for product ID, brand, and model
product_mapping = df0[['asin', 'brand', 'title']].copy()

# rename the columns to match the desired format
product_mapping = product_mapping.rename(
    columns={'asin': 'product_id', 'brand': 'product_brand', 'title': 'product_model_name'})

# save the product mapping as a CSV file
product_mapping.to_csv('product_mapping.csv', index=False)

In [34]:
df1 = pd.read_csv('product_mapping.csv')

In [35]:
df2= pd.read_csv('20191226-reviews.csv')

In [36]:
df2.columns

Index(['asin', 'name', 'rating', 'date', 'verified', 'title', 'body',
       'helpfulVotes'],
      dtype='object')

In [37]:
# Drop 'asin', 'name', 'rating', 'date', 'verified', 'helpfulVotes'
df2.drop(['asin', 'name', 'rating', 'date', 'verified', 'helpfulVotes'], axis=1, inplace=True)

In [38]:
df2.head()

,title,body
0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


In [39]:
df1.columns

Index(['product_id', 'product_brand', 'product_model_name'], dtype='object')

In [40]:
# Save the new process dataframe as a CSV file
df2.to_csv('processed_reviews.csv', index=False)

In [41]:
#load the processed dataframe
df3 = pd.read_csv('processed_reviews.csv')

In [42]:
df3.head()

,title,body
0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


### Installing reguired libraries

In [14]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.4 MB/s eta 0:00:00


## BERT model is used to predict the sentiment of the review.

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [17]:
# Define the device to use for training (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [19]:
# Load the pre-trained BERT-SA model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at